# SUP ML 3 - PREDICT

# Librerias

In [79]:
# Datos
import numpy as np
import pandas as pd
import pickle

# Preprocessing
from unidecode import unidecode
from sklearn.preprocessing import LabelEncoder

# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

# inhabilita warnings
import warnings
warnings.filterwarnings("ignore")

# Carga modelo

In [80]:
model = pickle.load(open('C:/Users/aalei/Desktop/Marzo 2024/ENTREGA2_COPY/model/here_you_save_model_and_preprocessors.txt', 'rb'))
print('MODEL: CatBoostClassifier','\n','Model features list:','\n',model.feature_names_)

MODEL: CatBoostClassifier 
 Model features list: 
 ['rev', 'mou', 'totmrc', 'change_mou', 'drop_vce', 'drop_dat', 'blck_vce', 'blck_dat', 'unan_vce', 'unan_dat', 'plcd_dat', 'recv_vce', 'recv_sms', 'custcare', 'ccrndmou', 'threeway', 'mou_cvce', 'mou_cdat', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'peak_vce', 'peak_dat', 'mou_peav', 'mou_pead', 'opk_vce', 'opk_dat', 'mou_opkv', 'drop_blk', 'complete', 'callfwdv', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'new_cell', 'asl_flag', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3mou', 'avg3qty', 'avg3rev', 'avg6rev', 'hnd_price', 'lor', 'adults', 'income', 'numbcars', 'eqpdays', 'num_kids', 'crclscod_A', 'crclscod_B', 'crclscod_C', 'crclscod_D', 'crclscod_E', 'crclscod_OTHER', 'crclscod_Z', 'prizm_social_one_C', 'prizm_social_one_R', 'prizm_social_one_S', 'prizm_social_one_T', 'prizm_social_one_U', 'prizm_social_one_UNKNOWN', 'area_ATLANTIC SOUTH', 'area_CALIFORNIA NORTH', 'area_CENTRAL/SOUTH

# Carga PREDICT dataset

In [81]:
sample_predict = pd.read_csv('C:/Users/aalei/Desktop/Marzo 2024/ENTREGA2_COPY/data/telecom_churn_PREDICT.csv') #,index_col=0
print('Rows: ', sample_predict.shape[0], '  Columns: ', sample_predict.shape[1], '\n')
sample_predict.head(2)

Rows:  10000   Columns:  99 



,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,...,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,30.8350,136.75,29.99,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,...,1.0,U,U,U,U,U,U,Y,216.0,1090001
1,35.8475,352.75,24.27,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,...,0.0,N,U,U,U,U,U,N,101.0,1090002


# ML Preprocessing

In [82]:
# Checkear duplicados en Customer_ID
print('Duplicated rows in Customer_ID:', sample_predict['Customer_ID'].duplicated().sum())
# Columna Customer_ID transformada a índice
df=sample_predict.set_index('Customer_ID')
print('Updated index to Customer_ID')
print('Rows: ', df.shape[0], '  Columns: ', df.shape[1])
df.head(2)

Duplicated rows in Customer_ID: 0
Updated index to Customer_ID
Rows:  10000   Columns:  98


,rev,mou,totmrc,da,ovrmou,ovrrev,vceovr,datovr,roam,change_mou,...,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays
Customer_ID,,,,,,,,,,,,,,,,,,,,,
1090001,30.8350,136.75,29.99,0.2475,1.25,0.500,0.5,0.000,0.0975,48.25,...,NaN,1.0,U,U,U,U,U,U,Y,216.0
1090002,35.8475,352.75,24.27,0.4950,23.25,9.285,8.7,0.585,1.8000,-352.75,...,NaN,0.0,N,U,U,U,U,U,N,101.0


In [83]:
# Eliminar columnas con valor único 
del(df['infobase'])
print(f'Column Infobase was deleted')
print('N° Rows: ',df.shape[0], '  Columns: ', df.shape[1])

Column Infobase was deleted
N° Rows:  10000   Columns:  97


In [84]:
# Clasificar variables en numéricas y categóricas
var_cat = df.select_dtypes(include=['object']).columns.tolist()
var_num = df.select_dtypes(include=['int','float']).columns.tolist()
print('Total var_cat:', len(var_cat), '   Total var_num:', len(var_num), '    Total variables:', len(var_cat)+len(var_num))

Total var_cat: 20    Total var_num: 77     Total variables: 97


In [85]:
# Para que todo lo que entren (datos del test, datos nuevos tengan las mismas características), Eliminar los tildes:
df[var_cat] = df[var_cat].apply(lambda x: x.apply(lambda y: unidecode(y) if isinstance(y, str) else y))
print('Accent mark deleted', '\n')

# Transformar todos los str a letras mayúsculas
df[var_cat] = df[var_cat].apply(lambda x: x.str.upper() if x.dtype == 'O' else x)
print('Transformed to uppercase', '\n')

# Eliminar ' area' al final de cada string en la columna area
df['area'] = df['area'].str.replace(' AREA', '')
print('AREA word removed', '\n')

# Cambiar 'UNKW' por 'UNKNOWN', ya que este será un valor a imputar en nulos posteriormente
df['hnd_webcap'] = df['hnd_webcap'].replace('UNKW','UNKNOWN')
print('UNKNOWN instead UNKW in hnd_webcap category', '\n')

# Reemplazar la categoría 'T' por 'U' en la variable dualband
df['dualband'] = df['dualband'].str.replace('T', 'U')
df['dualband'] = df['dualband'].str.replace('U', 'UNKNOWN')
print('U instead T in dualband category', '\n')

Accent mark deleted

 

Transformed to uppercase 

AREA word removed 

UNKNOWN instead UNKW in hnd_webcap category 

U instead T in dualband category 



In [86]:
negative_values = ['rev','totmrc','avg6rev','eqpdays']
df[negative_values] = df[negative_values].applymap(lambda x: -9999 if x < 0 else x)

In [87]:
# Se agrupan las columnas kids in num_kids
list_to_kids=['kid0_2','kid3_5','kid6_10','kid11_15','kid16_17']
for i in list_to_kids:
    df[i] = df[i].replace({'U': 0, 'Y': 1})

df['num_kids'] = df[list_to_kids].sum(axis=1)

drop_cols = list(set(list_to_kids))
df.drop(columns=drop_cols, inplace=True)   

var_cat = df.select_dtypes(include=['object']).columns.tolist()
var_num = df.select_dtypes(include=['int','float']).columns.tolist()
print('Total var_cat:', len(var_cat), '   Total var_num:', len(var_num), '    Total variables:', len(var_cat)+len(var_num))

Total var_cat: 15    Total var_num: 78     Total variables: 93


In [88]:
#Dejar la variable Credit class code todas con una sola letra
df['crclscod'] = df['crclscod'].apply(lambda x: x[0] if x else x)
print('crslscod modify to one letter instead two', '\n')

# Agrupar categorias en las variables categoricas
list_to_group = ['ethnic', 'crclscod']
grouping_dict = {
    'crclscod': ['A', 'B', 'C', 'E', 'D', 'Z'],
    'ethnic': ['N', 'H', 'S', 'U', 'G','Z','O','I','J','F','B']
}
for column in list_to_group:
    no_group_values = grouping_dict.get(column, [])
    df[column] = df[column].apply(lambda x: 'OTHER' if x not in no_group_values and not pd.isna(x) else x)
print(f'Categories were grouped for: {list_to_group}\n') 

var_cat = df.select_dtypes(include=['object']).columns.tolist()

crslscod modify to one letter instead two 

Categories were grouped for: ['ethnic', 'crclscod']



In [89]:
# Se imputan los nulos como 'UNKNOWN'
var_cat = df.select_dtypes(include=['object']).columns.tolist()
for i in var_cat:
    df[i].fillna('UNKNOWN', inplace=True)
print('Null values transformed to UNKNOWN')

Null values transformed to UNKNOWN


In [90]:
# Columnas numéricas se reemplazan con -9999
for i in var_num:
   df[i].fillna(-9999, inplace = True)
print('Null values transformed to -9999')

Null values transformed to -9999


In [91]:
# Check Nulos
df.isnull().sum().sum()

0

In [92]:
# Si tienen unicamente 2 clases, N: 0, Y: 1 y U or UNKNOWN: -9999
mapeo_asl = {'N': 0, 'Y': 1}
df['asl_flag'] = df['asl_flag'].replace(mapeo_asl)
print(f'Unique values for asl_flag: {df['asl_flag'].unique()}')

mapeo_newcell = {'N': 0, 'Y': 1, 'U': -9999}
df['new_cell'] = df['new_cell'].replace(mapeo_newcell)
print(f'Unique values for new_cell: {df['new_cell'].unique()}')

# Actualizar variables en numérica y categóricas
var_cat = df.select_dtypes(include=['object']).columns.tolist()
var_num = df.select_dtypes(include=['int', 'float']).columns.tolist()
print('Total var_cat:', len(var_cat), '   Total var_num:', len(var_num), '    Total variables:', len(var_cat)+len(var_num))

Unique values for asl_flag: [0 1]
Unique values for new_cell: [-9999     1     0]
Total var_cat: 13    Total var_num: 80     Total variables: 93


In [93]:
# onehotencoding
df = pd.get_dummies(df, columns=var_cat)
print('Rows: ', df.shape[0], '  Columns: ', df.shape[1])
# Actualizar variables en numérica y categóricas
var_cat = df.select_dtypes(include=['object']).columns.tolist()
var_num = df.select_dtypes(include=['int', 'float']).columns.tolist()
print('Total var_cat:', len(var_cat), '   Total var_num:', len(var_num), '    Total variables:', len(var_cat)+len(var_num))

Rows:  10000   Columns:  171
Total var_cat: 0    Total var_num: 80     Total variables: 80


In [94]:
cols_to_drop = ['totmou', 'mou_opkd', 'da', 'phones', 'ovrmou', 'datovr', 'rv', 'models', 'vceovr', 'plcd_vce', 'cc_mou', 'totrev', 'change_rev', 'comp_vce', 'inonemin', 'attempt', 'refurb_new_N', 'avg6qty', 'roam', 'avg6mou', 'truck', 'marital_UNKNOWN', 'ovrrev', 'creditcd_Y', 'comp_dat', 'ethnic_UNKNOWN', 'ownrent_UNKNOWN', 'totcalls', 'forgntvl','ownrent_R', 'HHstatin_G', 'HHstatin_H', 'dwllsize_C', 'dwllsize_D', 'dwllsize_E', 'dwllsize_F', 'dwllsize_G', 'dwllsize_H', 'dwllsize_I', 'dwllsize_J', 'dwllsize_K', 'dwllsize_L', 'dwllsize_M', 'dwllsize_N', 'dwllsize_O', 'ethnic_B', 'creditcd_UNKNOWN']
df.drop(columns=cols_to_drop, inplace=True)

In [95]:
sample_predict = df.copy()

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [96]:
# Check features matched features model
features_predict = list(sample_predict.columns)
model_features = list(model.feature_names_)

In [97]:
print('Columnas en dataset predict:',len(features_predict))
print('Variables en modelos:',len(model_features))
print('¿Match?:', model_features == features_predict)

Columnas en dataset predict: 124
Variables en modelos: 124
¿Match?: True


In [98]:
missing_features = [i for i in model_features if i not in features_predict]
print('Variables que faltan el el dataset:\n', missing_features)

Variables que faltan el el dataset:
 []


In [99]:
print(model_features)
print(features_predict)

['rev', 'mou', 'totmrc', 'change_mou', 'drop_vce', 'drop_dat', 'blck_vce', 'blck_dat', 'unan_vce', 'unan_dat', 'plcd_dat', 'recv_vce', 'recv_sms', 'custcare', 'ccrndmou', 'threeway', 'mou_cvce', 'mou_cdat', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'peak_vce', 'peak_dat', 'mou_peav', 'mou_pead', 'opk_vce', 'opk_dat', 'mou_opkv', 'drop_blk', 'complete', 'callfwdv', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'new_cell', 'asl_flag', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3mou', 'avg3qty', 'avg3rev', 'avg6rev', 'hnd_price', 'lor', 'adults', 'income', 'numbcars', 'eqpdays', 'num_kids', 'crclscod_A', 'crclscod_B', 'crclscod_C', 'crclscod_D', 'crclscod_E', 'crclscod_OTHER', 'crclscod_Z', 'prizm_social_one_C', 'prizm_social_one_R', 'prizm_social_one_S', 'prizm_social_one_T', 'prizm_social_one_U', 'prizm_social_one_UNKNOWN', 'area_ATLANTIC SOUTH', 'area_CALIFORNIA NORTH', 'area_CENTRAL/SOUTH TEXAS', 'area_CHICAGO', 'area_DALLAS', 'area_DC/MA

# PREDICT

* predict() y predict_proba()

In [100]:
# Reordena variables
X_pred = sample_predict[model_features]

In [101]:
predictions = model.predict(X_pred)

In [102]:
predict = model.predict(sample_predict)[1]

In [103]:
predict_proba = model.predict_proba(sample_predict)[:,1][1]

In [104]:
print('Clientes que van a abandonar próximo mes (churn=1)',predict)
print('Probabilidad de abandono: {:.2%}'.format(predict_proba))

Clientes que van a abandonar próximo mes (churn=1) 0
Probabilidad de abandono: 34.79%


# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [105]:
df_predict= pd.DataFrame(predictions, columns=['churn'], index = X_pred.index)

In [107]:
df_predict.to_csv('C:/Users/aalei/Desktop/Marzo 2024/ENTREGA2_COPY/data/telecom_churn_PREDICTIONS.csv')